In [1]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install wandb

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None # None for auto detection.
load_in_4bit = True # Use 4bit quantization to reduce memory usage.


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit, #qlora
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9.post3: Fast Llama patching. Transformers = 4.45.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.9.post3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["Instruction"]
    inputs       = examples["Input"]
    outputs      = examples["Response"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("Ivanich/datafeeling_posts", split = "train")

dataset = dataset.map(formatting_prompts_func, batched = True,)


In [6]:
from pprint import pprint
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Write post about the following topic: ", # instruction
        "Как запустить gen ai стартап", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
pprint(tokenizer.batch_decode(outputs))

['<|begin_of_text|>Below is an instruction that describes a task, paired with '
 'an input that provides further context. Write a response that appropriately '
 'completes the request.\n'
 '\n'
 '### Instruction:\n'
 'Write post about the following topic: \n'
 '\n'
 '### Input:\n'
 'Как запустить gen ai стартап\n'
 '\n'
 '### Response:\n'
 'Запускать gen ai стартапы сейчас - это довольно сложно, так как генетические '
 'алгоритмы и искусственный интеллект - это довольно сложные и не совсем '
 'понятные технологии. Однако, есть несколько способов, которые могут помочь '
 'вам запустить gen ai стартап.\n'
 '\n'
 'Первый способ - это найти компанию, которая уже занимается разработкой и '
 'внедрением gen ai технологий и предложить им сотрудничество. В этом случае '
 'вы можете получить доступ к их знаниям и опыту, а также использовать их '
 'инфраструктуру']


In [5]:
import wandb
from google.colab import userdata
wandb.login(key=userdata.get('WB_KEY'))

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: hvizze (misis_edu). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to="wandb"
    ),
)

Map (num_proc=2):   0%|          | 0/587 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [7]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.984 GB of memory reserved.


In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 587 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
1,2.797100
2,2.848000
3,2.872600
4,3.002400
5,3.009400
6,2.616800
7,2.426500
8,2.370900
9,2.451200
10,2.526100


In [9]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

617.8073 seconds used for training.
10.3 minutes used for training.
Peak reserved memory = 8.307 GB.
Peak reserved memory for training = 2.323 GB.
Peak reserved memory % of max memory = 56.326 %.
Peak reserved memory for training % of max memory = 15.751 %.


In [10]:
from pprint import pprint
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Write post about the following topic: ", # instruction
        "как запустить gen ai стартап", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
pprint(tokenizer.batch_decode(outputs))

['<|begin_of_text|>Below is an instruction that describes a task, paired with '
 'an input that provides further context. Write a response that appropriately '
 'completes the request.\n'
 '\n'
 '### Instruction:\n'
 'Write post about the following topic: \n'
 '\n'
 '### Input:\n'
 'как запустить gen ai стартап\n'
 '\n'
 '### Response:\n'
 '🤔 Как запустить gen ai стартуп?\n'
 '\n'
 '🧠 В прошлой статье я описал, как за 4 месяца я запустил и продал gen ai '
 'стартап. В этой статье я хочу поделиться, как это все делалось. \n'
 '\n'
 '🤖 Погнали!\n'
 '\n'
 '🔥 Первый шаг - собрать команду. В команду я набрал 3 человека. Это был мой '
 'старший брат и 2 моих друзей. Мы быстро сформировали команду и начали '
 'работу. \n'
 '\n'
 '🤹\u200d♀️ Второй шаг']
